In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mglearn

문자열로 된 데이터 종류
- 범주형 데이터 (ex. 색상, 성별 etc.)
- 범주에 연결할 수 있는 임의의 문자열 (ex. 다홍색(비슷한 종류) >>> 빨강, 검점색(오타) >>> 검정)
- 구조화된 문자열 데이터 (ex. 도로명주소)
- 말 그대로 문자열 (ex. .txt 파일)

In [4]:
!tree data/aclImdb

폴더 PATH의 목록입니다.
볼륨 일련 번호가 0000002F 0639:BF7F입니다.
C:\USERS\RKSLA\DOCUMENTS\PYTHON\BASIC\ML\PPB\DATA\ACLIMDB
├─test
│  ├─neg
│  └─pos
└─train
    ├─neg
    └─pos


In [3]:
from sklearn.datasets import load_files

reviews_train = load_files("data/aclImdb/train/")
text_train, y_train = reviews_train.data, reviews_train.target

print("TYPEOF text_train: ", type(text_train))
print("SIZEOF text_train: ", len(text_train))
print("text_train[6]: \n ", text_train[6])

TYPEOF text_train:  <class 'list'>
SIZEOF text_train:  25000
text_train[6]: 
  b"This movie has a special way of telling the story, at first i found it rather odd as it jumped through time and I had no idea whats happening.<br /><br />Anyway the story line was although simple, but still very real and touching. You met someone the first time, you fell in love completely, but broke up at last and promoted a deadly agony. Who hasn't go through this? but we will never forget this kind of pain in our life. <br /><br />I would say i am rather touched as two actor has shown great performance in showing the love between the characters. I just wish that the story could be a happy ending."


In [5]:
text_train = [doc.replace(b"<br />", b"") for doc in text_train]

In [6]:
print("클래스별 샘플 수 (TRAIN): ", np.bincount(y_train))

클래스별 샘플 수 (TRAIN):  [12500 12500]


In [7]:
reviews_test = load_files("data/aclImdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target

print("테스트 데이터의 문서 수: ", len(text_test))
print("클래스별 샘플 수 (TEST): ")

테스트 데이터의 문서 수:  25000
클래스별 샘플 수 (TEST): 


In [8]:
text_test = [doc.replace(b"<br />", b"") for doc in text_test]

# BOW (Bag of Words)

과정
1. 토큰화: 각 문서를 문서에 포함된 단어(토큰)로 나눈다. 이 때 기준은 띄어쓰기, 점 따위로 한다
2. 어휘 사전 구축: 모든 문서에 나타난 모든 단어의 어휘를 모으고 번호를 매긴다. 순서는 알파벳 순으로 한다
3. 인코딩: 어휘 사전의 단어가 문서마다 몇 번이나 나타나는지를 센다


In [10]:
bards_words = [
    "the fool doth think he is wise, ", 
    "but the wise man knows himself to be a fool"
]

In [20]:
"""
    feature_extraction.text.CountVectorizer: sklearn의 BOW 변환 도구
    
    상세 동작
        1. vect.fit(문자열) 결과는 다음과 같다
            1.1 전달한 문자열을 단어 경계 단위로 나눈다. 이 때 모든 단어는 소문자로 바꾼다
            1.2 추출한 단어 목록의 unique 목록을 만들어 알파벳 순으로 정렬한다
            
        2. vect.transform(문자열) 의 결과는 다음과 같다
            2.1 문자열을 희소 행렬로 바꾼다
            2.2 ndarray일 때 값은 전달한 문자열에 해당 위치 단어가 몇 회 출현하는지에 대한 횟수다
"""

from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
vect.fit(bards_words)

CountVectorizer()

In [12]:
"""
    vect.vocabulary_: BOW에 있는 단어 종류를 사전 형식으로 나타냄
"""

print("어휘 사전의 크기: ", len(vect.vocabulary_))
print("어휘 사전의 내용: \n", vect.vocabulary_)

어휘 사전의 크기:  13
어휘 사전의 내용: 
 {'the': 9, 'fool': 3, 'doth': 2, 'think': 10, 'he': 4, 'is': 6, 'wise': 12, 'but': 1, 'man': 8, 'knows': 7, 'himself': 5, 'to': 11, 'be': 0}


In [13]:
bag_of_words = vect.transform(bards_words)
print("BOW: ", repr(bag_of_words))

BOW:  <2x13 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>


In [14]:
print("BOW TO NDARRAY: \n", bag_of_words.toarray())

BOW TO NDARRAY: 
 [[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


In [15]:
wrds = pd.Series(bards_words[0].split(" ")[ : -1])
wrds

0      the
1     fool
2     doth
3    think
4       he
5       is
6    wise,
dtype: object

In [16]:
wrds[6] = wrds[6][ : -1]
wrds

0      the
1     fool
2     doth
3    think
4       he
5       is
6     wise
dtype: object

In [17]:
print(wrds.sort_values().to_list())
print(bag_of_words.toarray()[0])
print(vect.vocabulary_)

['doth', 'fool', 'he', 'is', 'the', 'think', 'wise']
[0 0 1 1 1 0 1 0 0 1 1 0 1]
{'the': 9, 'fool': 3, 'doth': 2, 'think': 10, 'he': 4, 'is': 6, 'wise': 12, 'but': 1, 'man': 8, 'knows': 7, 'himself': 5, 'to': 11, 'be': 0}


In [19]:
v2 = CountVectorizer()
v2.fit(["what is your major malfunction"])

v2.transform(['what you think is what you can be']).toarray()

array([[1, 0, 0, 2, 0]], dtype=int64)

In [21]:
kor_wrds = ["안녕하세요 저는 트위치에서 방송을 하고 있는 스트리머"]

kv = CountVectorizer()
kv.fit(kor_wrds)

CountVectorizer()

In [22]:
print("어휘 사전의 크기: ", len(kv.vocabulary_))
print("어휘 사전의 내용: \n", kv.vocabulary_)

어휘 사전의 크기:  7
어휘 사전의 내용: 
 {'안녕하세요': 2, '저는': 4, '트위치에서': 5, '방송을': 0, '하고': 6, '있는': 3, '스트리머': 1}


In [23]:
kr_bows = vect.transform(kor_wrds)
print("KR_BOW: ", repr(kr_bows))

KR_BOW:  <1x13 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>


In [24]:
print("KR_BOW TO NDARRAY: \n", kr_bows.toarray())

KR_BOW TO NDARRAY: 
 [[0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [25]:
vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)
print("X_train: \n", repr(X_train))

X_train: 
 <25000x75911 sparse matrix of type '<class 'numpy.int64'>'
	with 3431163 stored elements in Compressed Sparse Row format>


In [26]:
feature_names = vect.get_feature_names_out()

print("특성 개수: ", len(feature_names))
print("특성 [0 : 20]: ", feature_names[ : 20])
print("특성 [20010 : 20030]: ", feature_names[20010 : 20030])
print("특성 [ : : 2000]: ", feature_names[ : : 2000])

특성 개수:  75911
특성 [0 : 20]:  ['00' '000' '0000000000001' '00001' '00015' '000s' '001' '003830' '006'
 '007' '0079' '0080' '0083' '0093638' '00am' '00pm' '00s' '01' '01pm' '02']
특성 [20010 : 20030]:  ['doppelgangers' 'doppelgänger' 'dopplebangers' 'doppleganger' 'doppler'
 'dopy' 'doqui' 'dor' 'dora' 'dorado' 'dorama' 'doran' 'dorcas' 'dorcey'
 'dorday' 'dore' 'doreen' 'doremus' 'doren' 'dorf']
특성 [ : : 2000]:  ['00' 'adultery' 'appearence' 'bang' 'blissfully' 'burress' 'chaulk'
 'compensations' 'crossfire' 'derboiler' 'dop' 'empty' 'falling' 'formats'
 'gisbourne' 'hallucinogenics' 'honore' 'inferenced' 'johnston' 'laconic'
 'looping' 'matlin' 'modelling' 'nerdish' 'orion' 'periphery' 'preached'
 'raha' 'resolving' 'salli' 'shaffer' 'smurfs' 'stereotypic'
 'synchronized' 'tirard' 'una' 'venantini' 'whycome']


#### 특성 추출 개선 전 rough data 그대로 썼을 때의 결과

특성 추출 전에는 단수형/복수형을 서로 다른 단어로 인식하게되어 성능 저하가 발생할 수 있다

In [27]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

scores = cross_val_score(LogisticRegression(max_iter=1000), X_train, y_train, n_jobs=-1)
print("MEAN(CROSS_VAL): {:.2f}".format(np.mean(scores)))

MEAN(CROSS_VAL): 0.88


In [28]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [10**x for x in range(-3, 2)]}

grid = GridSearchCV(LogisticRegression(max_iter=5000), param_grid, n_jobs=-1)
grid.fit(X_train, y_train)
print("BEST CROSS VAL SCORE: {:.2f}".format(grid.best_score_))
print("BEST PARAMS: ", grid.best_params_)

BEST CROSS VAL SCORE: 0.89
BEST PARAMS:  {'C': 0.1}


In [29]:
X_test = vect.transform(text_test)
print("TEST_SCORE: {:.2f}".format(grid.score(X_test, y_test)))

TEST_SCORE: 0.88


In [31]:
np.mean(grid.cv_results_['mean_fit_time'])

38.32856240272522

### 특성 추출

#### 1. min_df: 최소 n개 문서에서 나타난 단어만 포함

In [32]:
vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)

print("min_df로 제한한 X_train: ", repr(X_train))

min_df로 제한한 X_train:  <25000x27264 sparse matrix of type '<class 'numpy.int64'>'
	with 3352876 stored elements in Compressed Sparse Row format>


In [33]:
feature_names = vect.get_feature_names_out()

print("FEATURE [ : 50] \n", feature_names[ : 50])
print("20010 ~ 20030 \n", feature_names[20010 : 20030])
print("[ : : 700] \n", feature_names[ : : 700])

FEATURE [ : 50] 
 ['00' '000' '007' '00s' '01' '02' '03' '04' '05' '06' '07' '08' '09' '10'
 '100' '1000' '100th' '101' '102' '103' '104' '105' '107' '108' '10s'
 '10th' '11' '110' '112' '116' '117' '11th' '12' '120' '12th' '13' '135'
 '13th' '14' '140' '14th' '15' '150' '15th' '16' '160' '1600' '16mm' '16s'
 '16th']
20010 ~ 20030 
 ['repetition' 'repetitions' 'repetitious' 'repetitive' 'rephrase'
 'replace' 'replaced' 'replacement' 'replaces' 'replacing' 'replay'
 'replayed' 'replaying' 'replays' 'replete' 'replica' 'replicas'
 'replicate' 'replied' 'replies']
[ : : 700] 
 ['00' 'affectionate' 'appropriate' 'barbet' 'blur' 'butch' 'cheery'
 'commit' 'courtroom' 'deconstruct' 'disgraceful' 'dvr' 'escort' 'fella'
 'freezing' 'gorillas' 'havana' 'hunk' 'insist' 'juice' 'left' 'mafia'
 'met' 'musicals' 'occult' 'parliament' 'pleasence' 'prop' 'recites'
 'revisionist' 'sassy' 'shedding' 'sniff' 'stems' 'swear' 'thugs' 'tween'
 'vanish' 'whale']


In [34]:
grid = GridSearchCV(LogisticRegression(max_iter=5000), param_grid, n_jobs=-1)
grid.fit(X_train, y_train)
print("BEST CROSS VAL SCORE: {:.2f}".format(grid.best_score_))

BEST CROSS VAL SCORE: 0.89


In [35]:
np.mean(grid.cv_results_['mean_fit_time'])

23.512445545196535

최소 출현 문서 수로 규제했을 때의 효과
1. 성능이 좋아진 것은 아니다
2. 대신 단어장 크기가 줄어 훈련 시간이 줄었다 (평균 38초 >>> 23초로 감소)

#### 2. 불용어 제외: 불필요한 관사, 조사, 전치사, 접속사 등은 사전에서 제외

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

print("SIZEOF STOPWORDS: ", len(ENGLISH_STOP_WORDS))
print(list(ENGLISH_STOP_WORDS)[ : : 10])

In [ ]:
vect = CountVectorizer(min_df=5, stop_word="english").fit(text_train)
X_train = vect.transform(text_train)
print(repr(X_train))

In [ ]:
grid = GridSearchCV(LogisiticRegression(max_iter=5000), param_grid, n_jobs=-1)
grid.fit(X_train, y_train)
print("BEST CROSS VAL SCORE: {:.2f}".format(grid.best_score_))

# tf-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression(max_iter=5000))
param_grid = {'logisticregression__C': [10**x for x in range(-3, 3)]}

grid = GridSearchCV(pipe, param_grid, n_jobs=-1)
grid.fit(text_train, y_train)
print("BEST CROSS VAL SCORE: {:.2f}".format(grid.best_score_))

In [ ]:
vectorizer = grid.best_estimator_.named_steps['tfidfvectorizer']

X_train = vectorizer.transform(text_train)
max_value = X_train.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
feature_names = np.array(vectorizer.get_feature_names_out())

print("LOW-tfidf FEATURES: \n", feature_naems[sorted_by_tfidf[ : 20]])
print("HIGH-tfidf FEATURES: \n", feature_naems[sorted_by_tfidf[-20 : ]])

In [ ]:
sorted_by_idf = np.argsort(vectorizer.idf_)
print("LOWEST-idf FEATURES: \n", feature_names[sorted_by_idf[ : 100]]

In [ ]:
mglearn.tools.visualize_coefficients(
    grid.best_estimator_.named_steps['logisticregression'].coef_[0],
    feature_names, n_top_features=40
)

In [ ]:
print('bards_words: \n', bards_words)

In [ ]:
cv = CounterVectrizer(ngram_range=(1, 1)).fit(bards_words)
print("어휘 사전 크기: ", len(cv.vocabulary_))
print(cv.get_features_names_out())

In [ ]:
cv = CounterVectrizer(ngram_range=(2, 2)).fit(bards_words)
print("어휘 사전 크기: ", len(cv.vocabulary_))
print(cv.get_features_names_out())

In [ ]:
cv = CounterVectrizer(ngram_range=(1, 3)).fit(bards_words)
print("어휘 사전 크기: ", len(cv.vocabulary_))
print(cv.get_features_names_out())

In [ ]:
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression(max_iter=5000))

param_grid = {
    'logisticregression__C': [10**x for x in range(-3, 3)],
    'tfidfVectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)]
}

grid = GridSearchCV(pipe, param_grid, n_jobs=-1)
grid.fit(text_train, y_train)

print("BEST CV SCORE: {:.2f}".format(grid.best_score_))
print("BEST PARAMS: ", grid.best_params_)

In [ ]:
scores = grid.cv_results_['mean_test_score'].reshape(-1, 3).T

heatmap = mglearn.tools.heatmap(
    scores,
    xlabel='C', ylabel='ngram_range',
    xticklabels=param_grid['logisticregression__C'], yticklabels=param_grid['tfidfvectorizer__ngram_range']
    cmap='viridis',
    fmt="%.3f",
)

plt.colorbar(heatmap)
plt.show()

In [ ]:
vect = grid.best_estimator_.named_steps['tfidfvectorizer']
feature_naes = np.array(vect.get_feature_names_out())
coef = grid.best_estimator_.naemd_steps['logistigregression'].coef_
mgelarn.tools.visualize_coefficients(coef[0], feature_names, n_top_features=40)

In [ ]:
mask = np.array([len(feature.split(" ")) for feature in feature_names]) == 3

mglearn.tools.visualize_coefficients(coef.ravel()[mask], feature_names[mask], n_top_features=40)

In [ ]:
import spacy
import nltk

nltk.__version__
spacy.__version__

In [ ]:
en_nlp=  spacy.load('en_core_web_sm')
stemmer = nltk.stem.PorterStemmer()

def compare_normalization(doc):
    doc_spacy = en_nlp(doc)
    print("표제어: ")
    print([token.lenna_ for token in doc_spacy])
    print("어간: ")
    print([stemmer.stem(token.norm_.lower()) for token in doc_spacy])

In [ ]:
compare_normalization(
    u"Our meeting today was worse than yesterday, "
    "I'm scared of meeting the clients tomorrow"
)

In [ ]:
en_nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def custom_tokenizer(document):
    doc_spacy = en_nlp(document)
    return [token.lemma for token in doc_spacy]

lemma_vect = CountVectoricer(tokenizer=custom_tokenizer, min_df=5)

In [ ]:
X_train_lemma = lemma_vect.fit_transform(text_train)
print("X_train_lemma.shape: ", X_train_lemma.shape)

In [ ]:
vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)
print("X_train.shape:", X_train.shape)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

param_grid = {'C': [10**x for x in range(-3, 3)]}
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.99, train_size=0.01, random_state=0)
grid = GridSearchCV(LogisticRegression(max_iter=5000), param_grid, cv=cv, n_jobs=-1)

grid.fit(X_train, y_train)
print("BEST CV SCORE(countervectorizer): ", grid.best_score_)

grid.fit(X_train_lemma, y_train)
print("BEST CV SCORE(lemma): ", grid.best_score_)

In [ ]:
vect = CountVectorizer(max_features=10000, max_df=0.15)
X = vect.fit_transform(text_train)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, learning_method='batch', max_iter=25, random_state=0, n_jobs=-1)

document_topics = lda.fit_transform(X)

In [ ]:
print("lda.components_.shape", lda.components_.shape)

In [ ]:
sorting = np.argsort(lda.components, axis=1)[ : , : : -1]
feature_names = np.array(vect.get_feature_names_out())

mglearn.tools.print_topics(
    topics=range(10),
    feature_names=feature_names, 
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10
)

In [ ]:
lda100 = LatentDirichletAllocation(
    n_components=100,
    learning_method='batch',
    max_iter=25,
    random_state=0, n_jobs=-1
)

document_topics100 = lda100.fit_transform(X)

In [ ]:
topics = np.array([7, 16, 24, 25, 28, 36, 37, 41, 45, 51, 53, 54, 63, 89, 97])

In [ ]:
sorting = np.argsort(lda100.components_, axis=1)[ : , : : -1]
feature_names = np.array(vect.get_feature_names_out())
mglearn.tools.print_topics(
    topics=topics, 
    feature_names=feature_names,
    sorting=sorting, topics_per_chunk=5, n_words=20
)

In [ ]:
music = np.argsort(document_topics100[ : , 45])[ : : -1]

for i in music[ : 10] :
    print(b".".join(text_train[i].split(b".")[ : 2]) + b".\n")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 10))

topic_names = ["{:>2} ".format(i) + " ".join(words) for i, words in enumerate(feature_names[sorting[ : , : 2]])]

forcol in [0, 1] :
    start = col * 50
    end = (col + ) * 50
    ax[col].barh(np.arange(50), np.sum(document_topics100, axis=0)[start : end])
    ax[col].set_yticks(np.arange(50))
    ax[col].set_yticklabels(topic_names[start : end], ha='left', va='top')
    ax[col].invert_yaxis()
    ax[col].set_xlim(0, 2000)
    yax = ax[col].get_yaxis()
    yax.set_tick_params(pad=130)
    
plt.tight_layout()